<a href="https://colab.research.google.com/github/mralamdari/NLP-Text_Classification/blob/main/NLP_Text_Classification_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import nltk
import numpy
import tensorflow as tf
import tarfile
import os
import wget
import sys
import warnings
from zipfile import ZipFile

warnings.filterwarnings('ignore')

# Data
###IMDB Dataset

In [4]:
%%capture
try:
    
    from google.colab import files
    
    !wget -P DATAPATH http://nlp.stanford.edu/data/glove.6B.zip
    !unzip DATAPATH/glove.6B.zip -d DATAPATH/glove.6B
    
    !wget -P DATAPATH http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
    !tar -xvf DATAPATH/aclImdb_v1.tar.gz -C DATAPATH
    
    BASE_DIR = 'DATAPATH'
    
except ModuleNotFoundError:
    
    if not os.path.exists('Data/glove.6B'):
        os.mkdir('Data/glove.6B')
        
        url='http://nlp.stanford.edu/data/glove.6B.zip'  
        wget.download(url,'Data')  
  
        temp='Data/glove.6B.zip' 
        file = ZipFile(temp)  
        file.extractall('Data/glove.6B') 
        file.close()
        
           
    if not os.path.exists('Data/aclImdb'):
        
        url='http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz' 
        wget.download(url,'Data')
        
        temp='Data/aclImdb_v1.tar.gz' 
        tar = tarfile.open(temp, "r:gz")
        tar.extractall('Data')      
        tar.close()
    
    BASE_DIR = 'Data'

In [5]:
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
TRAIN_DATA_DIR = os.path.join(BASE_DIR, 'aclImdb/train')
TEST_DATA_DIR = os.path.join(BASE_DIR, 'aclImdb/test')

In [6]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [8]:
def get_data(data_dir):
  texts = []
  labels_index = {'pos': 1, 'neg': 0}
  labels = []
  for name in sorted(os.listdir(data_dir)):
    path = os.path.join(data_dir, name)
    if os.path.isdir(path):
      if name=='pos' or name=='neg':
        label_id = labels_index[name]
        for fname in sorted(os.listdir(path)):
          fpath = os.path.join(path, fname)
          text = open(fpath, encoding='utf8').read()
          texts.append(text)
          labels.append(label_id)
  return texts, labels

#BERT Sentiment Classification
## IMDB ktrain